In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


In [ ]:

#define the paths
root_dir = "/content/gdrive/My Drive/Colab Notebooks/"
data_dir = root_dir + 'data/'
model_dir = root_dir + 'models/'

## Create a class model from keras


```
class SmallerVGGNet:

```




In [ ]:
# import the necessary packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
    
#########
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1
        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1
        # CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # (CONV => RELU) * 2 => POOL
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        # first (and only) set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))
        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        # return the constructed network architecture
        return model


In [ ]:
# import the necessary packages
# set the matplotlib backend so figures can be saved in the background
import matplotlib
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os


In [ ]:
#########
# initialize the initial learning rate, number of epochs to train for,
# and batch size
# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
#
#model_dir = 'models/'
#data_dir = 'data/'
EPOCHS = 100
INIT_LR = 1e-3
BS = 50
# initialize the data and labels
data = []
labels = []
#load images
print("[INFO] loading images...")
data_file = ['g1.p', 'g2.p', 'nada.p']
clases   = ['uno','dos','nada']
size = (224, 224,3)
# loop over the files
for fh,label in zip(data_file,clases):
    fh =  data_dir + fh
    matrix = pickle.load( open( fh , "rb" ) )
    for row in matrix:
        image = row.reshape(224,224,3)  
        #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (size[1], size[0]))
        # update the data and labels lists, respectively
        data.append(image)
        labels.append(label)
        
    matrix=None


data = np.array(data)/255.0
labels = np.array(labels)
ind=np.random.permutation(labels.size)
data=data[ind,:]
labels=labels[ind]
print("[INFO] data matrix: {:.2f}MB".format(data.nbytes / (1024 * 1000.0)))
print(data.shape)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
LB = lb.fit_transform(labels)


In [ ]:
#show some examples
plt.figure(figsize = (10,10))
plt.subplot(221)
index = 50
img = data[index] 
plt.imshow(img);    # plot the image
plt.axis('off');
plt.title(labels[index])

plt.subplot(222)
index = 300
img = data[index] 
plt.imshow(img);    # plot the image
plt.axis('off');
plt.title(labels[index])

plt.subplot(223)
index = 500
img = data[index] 
plt.imshow(img);    # plot the image
plt.axis('off');
plt.title(labels[index])

plt.subplot(224)
index = 700
img = data[index] 
plt.imshow(img);    # plot the image
plt.axis('off');
plt.title(labels[index])


In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
	LB, test_size=0.2, random_state=42, shuffle=True)
# construct the image generator for data augmentation
#aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
#	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
#	horizontal_flip=True, fill_mode="nearest")

aug = ImageDataGenerator( rotation_range=15,fill_mode="nearest")

# initialize the model
print("[INFO] compiling model...")
model = SmallerVGGNet.build(width=size[1], height=size[0],
	depth=size[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
# train the network
print("[INFO] training network...")
H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
 target_names=lb.classes_))

# compute the confusion matrix and and use it to derive the raw
# accuracy, sensitivity, and specificity
cm = confusion_matrix(testY.argmax(axis=1), predIdxs)
total = sum(sum(cm))

# show the confusion matrix
print(cm)



In [ ]:
# save the model to disk
print("[INFO] serializing network...")

file_name = model_dir + 'May_24_vgg_small.h5'

model.save(file_name, save_format='h5')

# save the label binarizer to disk
print("[INFO] serializing label binarizer...")
file_name = model_dir + 'hand_label_bin.p'

f = open(file_name, "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
lb.classes_